In [ ]:
from _global_vars import *
from clean import get_outliers_df, remove_outliers

In [ ]:
# read in cleaned df
sdf = spark.read.parquet('../data/curated/yt2019_feature_eng.parquet')
#sdf.show(1, vertical=True, truncate=100)

In [ ]:
# Note current record count
sdf.count()

---
## Erroneous record cleaning
---

### Filters (reflective of findings from univariate/preliminary analysis)
* Restrict range of passenger_count (1:6) [drop other records]
* Only take 312miles > trip distances > 0 (3 hours at maximum speed)
* Fares capped at $500 (0:500)
* remove trips with average speed over 104 mph and remove trips less than 1 mph
* Limit trip times to under 5 hours (300 minutes) (more than it takes to drive entire perimeter of nyc)
* Trips with a time of less than 2 minutes should be excluded => not very realistic => passenger may have cancelled
* Congestion surcharge capped at 10 usd -> any records beyond this can go away
* Ratecode == 1 and fare < 2.5

In [ ]:
# get & inspect outlier breakdown from helper function
outlier_df = get_outliers_df(sdf)
print(outlier_df.sort_values('prop_of_total', ascending=False))

In [7]:
# apply outlier detection to df
sdf = remove_outliers(sdf)
sdf.groupBy("is_valid_record").count().show()

In [ ]:
# remove bad records
sdf_filtered = sdf.filter(sdf.is_valid_record == True)
sdf_filtered.count()

In [ ]:
# inspect summary statistics
summary_stats_continuous = sdf_filtered.describe(*non_categorical_features)
summary_stats_continuous.show()

In [ ]:
sdf_filtered.write.mode('overwrite').parquet(export_relative_dir + "yt2019_cleaned.parquet")

In [ ]:
spark.sql("CLEAR CACHE")
spark.stop()